In [ ]:
import os
import sys
import gmsh
import copy
import numpy as np
import pandas as pd
import import_ipynb
import matplotlib.pyplot as plt
from pathlib import Path
from matplotlib.tri import Triangulation
from scipy.constants import mu_0, epsilon_0
from scipy.special import jvp, hankel2, h2vp, jv
from fem_pre_processing import read_mesh
from fem_processing import matrices_assembly as assembly
from fem_pos_processing import graph_results

## Physics Constants

In [3]:
OMEGA = 2 * np.pi * 3E8
K0 = OMEGA * np.sqrt(mu_0 * epsilon_0)
WAVELENGTH = 2 * np.pi / K0
OPERATIONS = {'real': np.real, 'imag': np.imag, 'abs': np.abs}

## `apply_physics()`

In [8]:
def apply_physics(FINITE_ELEMENT, mesh_data):
    """
    Adiciona uma nova chave 'source' a cada dicionário em conn_data.
    
    Parâmetros:
    - mesh_data: Dicionário contendo os dados da malha.
    - element_type: Tuple (tipo do elemento, ordem).
    
    Retorna:
    - mesh_data: O dicionário atualizado com a chave 'source' em cada elemento de conn_data.
    """

    # PML parameters
    R_COEFFICIENT = 1E-4                                    # Coeficiente de reflexão
    SIGMA_0X = -np.log(R_COEFFICIENT) / WAVELENGTH / K0     # Condutância de superfície
    x0 = 1                                                  # Posição do início da PML
    n = 1                                                   # Ordem polinomial da PML                            
    Sx, Sy = 1, 1                                           # Fator de escala da PML

    # Dictionary with all nodes in the mesh
    cell_data = mesh_data['cell']
    nodes_data = mesh_data['nodes']

    for key, cell in cell_data.items():  
        # Adicionar as propriedades do materiais ao dicionário da célula
        cell['stiffness_term'] = []
        cell['mass_term'] = []
        cell['source'] = []

        # Geometric data    
        xc = np.mean([mesh_data['nodes'][node]['xg'][0] for node in cell['conn']])
        yc = np.mean([mesh_data['nodes'][node]['xg'][1] for node in cell['conn']])
        Je = assembly.jacobian(FINITE_ELEMENT, mesh_data, cell, xik=(xc, yc))
        area = np.abs(np.linalg.det(Je)) * 0.5 
        cell['geo']['centroid'] = (xc, yc)        
        cell['geo']['jacobian'] = Je
        cell['geo']['area'] = area
        
        # Contorno do Espalhador
        gamma_s = {idx: node for idx, node in enumerate(cell['conn'])
                        if nodes_data[node]['bc']['tag'] == 101}
        if len(gamma_s) > 1:
            cell_data[key]['contour'] = {'type': 'Neumann', 'conn_dict': gamma_s}

        # Material properties Sx, Sy
        sigma_x = SIGMA_0X * (np.abs(xc) - x0) ** n
        sigma_y = SIGMA_0X * (np.abs(yc) - x0) ** n
        if cell['material']['name'] in ['PML_a', 'PML_b', 'PML_c', 'PML_d']:
            Sx = 1 - 1j * sigma_x
            Sy = 1 - 1j * sigma_y

        elif cell['material']['name'] in ['PML_I', 'PML_III']:
            Sy = 1 - 1j * sigma_y

        elif cell['material']['name'] in ['PML_II', 'PML_IV']:
            Sx = 1 - 1j * sigma_x

        # Stiffness matrix
        # ka = np.array([[Sy / Sx, 0], [0, Sx / Sy]])
        cell['material']['Sx_Sy'] = (Sx, Sy)
            
    # Adiciona os potenciais de Dirichlet
    for node in nodes_data.values():
        if node['bc']['type'] == 'Dirichlet' and node['bc']['value'] == None:
            # Atualizar o valor de Dirichlet para o nó
            node['bc']['value'] = -np.exp(-1j * K0 * node['xg'][0])
    
    return mesh_data

# `apply_pml_physics()`

In [ ]:
def apply_pml_physics(FINITE_ELEMENT, PML_DESIGN, mesh_data):
    # Dictionary with all nodes in the mesh
    cell_data = mesh_data['cell']
    nodes_data = mesh_data['nodes']

    # Parâmetros da PML
    x0 = PML_DESIGN['x0']       # Interface do PML
    n = PML_DESIGN['n']         # Ordem do PML 
    R = PML_DESIGN['R']         # Coeficiente de reflexão do truncamento
    SIGMA_0X = -np.log(R) / WAVELENGTH
    print(f"Sigma_0X: {SIGMA_0X}")

    # Adicionar as propriedades do materiais ao dicionário da célula
    for cell in cell_data.values():  
        # Geometric data    
        xc = np.mean([nodes_data[node]['xg'][0] for node in cell['conn']])
        yc = np.mean([nodes_data[node]['xg'][1] for node in cell['conn']])
        Je = assembly.jacobian(FINITE_ELEMENT, mesh_data, cell, xik=(xc, yc))
        area = np.abs(np.linalg.det(Je)) * 0.5 
        
        # Atualizar as propriedades da célula
        cell['geo']['centroid'] = (xc, yc)        
        cell['geo']['jacobian'] = Je
        cell['geo']['area'] = area

        # PML x-properties
        if abs(xc) < x0:
            Sx = 1
        else:
            Sx = 1 - 1j * SIGMA_0X * (abs(xc) - x0) ** n 

        # PML y-properties
        if abs(yc) < x0:
            Sy = 1
        else:
            Sy = 1 - 1j * SIGMA_0X * (abs(yc) - x0) ** n 
        
        # Material properties
        pml_tensor = np.array([[Sy / Sx, 0], [0, Sx / Sy]])
        pml_gamma = Sx * Sy

        # Material properties
        er = cell['material']['relative_electric_permittivity']
        mur = cell['material']['relative_magnetic_permeability']
        # sigma = cell['material']['electric_conductivity']
        # erc = er - 1j * sigma / (OMEGA * epsilon_0)
        # u_inc = np.exp(-1j * K0 * xc)

        # Adicionar as propriedades do materiais ao dicionário da célula
        cell['material']['(Sx, Sy)'] = (Sx, Sy)
        cell['p(x)'] = (1/mur) * pml_tensor
        cell['q(x)'] = -K0**2 * er * pml_gamma
        cell['f(x)'] = 0 * pml_gamma
        # cell['f(x)'] = K0**2 * (erc - 1) * u_inc * pml_gamma
        # cell['f(x)'] = -(K0**2 * cell['p(x)'] + cell['q(x)']) * u_inc * pml_gamma

        # Inicializa os contornos do domínio
        gamma_d = {}
        for idx, node in enumerate(cell['conn']):
            bc_type = nodes_data[node]['bc']['type']

            if bc_type == 'Dirichlet':
                gamma_d[idx] = node
                # Atualiza o valor da condição de contorno para 'gamma_d'
                nodes_data[node]['bc']['value'] = -np.exp(-1j * K0 * nodes_data[node]['xg'][0])
                
                # Dirichlet homogêneo no truncamento da PML
                # nodes_data[node]['bc']['value'] = 0
            
        # Define o contorno do elemento, caso aplicável
        if gamma_d:
            cell['contour'] = {'type': 'gamma_d', 'conn_dict': gamma_d}

    return mesh_data

## `create_p1_domain()`

In [3]:
def create_p1_domain(FINITE_ELEMENT, BOUNDARY, MATERIAL, DOMAIN_KEY, auto_save=True, view_mesh=False):
    # Parâmetros do elemento finito
    ElementType, ElementOrder = FINITE_ELEMENT
    mesh_data = {}

    # Dimensões do domínio 
    h = DOMAIN_KEY['h']     # Tamanho do elemento
    L = DOMAIN_KEY['L']     # Lado do retângulo externo
    ra = DOMAIN_KEY['ra']   # Raio do furo circular
    x0 = DOMAIN_KEY['x0']   # Lado do retângulo interno
    y0 = x0
    
    # Inicializar o Gmsh
    gmsh.initialize()
    gmsh.model.add("rectangular_pml")
    factory = gmsh.model.occ

    # Criar as superfícies retangulares externa e interna
    rect_outer = factory.addRectangle(-(x0 + L), -(y0 + L), 0, 2 * (x0 + L), 2 * (y0 + L))
    rect_inner = factory.addRectangle(-x0, -y0, 0, 2 * x0, 2 * y0)
    
    disk = factory.addDisk(0, 0, 0, ra, ra)
    outDimTags_plm_omega, _ = factory.cut([(2, rect_outer)], [(2, rect_inner)], removeTool=False)
    outDimTags_fs_omega, _ = factory.cut([(2, rect_inner)], [(2, disk)], removeTool=True)

    # Sincronizar após o corte do retângulo interno
    factory.synchronize()

    # Obter os contornos (curvas, dim=1) de cada superfície
    boundary_pml_ext = gmsh.model.getBoundary(outDimTags_plm_omega, oriented=True, recursive=False)
    boundary_pml_inn = gmsh.model.getBoundary(outDimTags_fs_omega, oriented=True, recursive=False)

    # Exibir os TAGs das curvas associadas a cada contorno
    tagList_scatterer = [-tag[1] for tag in boundary_pml_inn if tag[1] < 0]
    tagList_pml_inn = [tag[1] for tag in boundary_pml_inn if tag[1] > 0]
    tagList_pml_ext = [tag[1] for tag in boundary_pml_ext if tag[1] > 0]

    # Adicionar grupos físicos para curvas (Dim=1)
    for i, CurveTagList in enumerate([tagList_scatterer]):
        gmsh.model.addPhysicalGroup(1, CurveTagList, tag=BOUNDARY[i]['tag'], name=BOUNDARY[i]['name'])
    
    # Adicionar grupos físicos para superfícies (Dim=2)	    
    for i, SurfaceList in enumerate([outDimTags_plm_omega, outDimTags_fs_omega]):
        SurfaceTagList = [DimTag[1] for DimTag in SurfaceList]
        gmsh.model.addPhysicalGroup(2, SurfaceTagList, tag=MATERIAL[i]['tag'], name=MATERIAL[i]['name'])

    # Definir ordem dos elementos
    gmsh.option.setNumber("Mesh.MeshSizeMax", h)
    gmsh.option.setNumber("Mesh.MeshSizeMin", h)
    gmsh.model.mesh.generate(2)
    gmsh.model.mesh.setOrder(ElementOrder)

    # Visualizar a malha no ambiente Gmsh (opcional)
    if view_mesh:
        gmsh.fltk.run()
    
    if auto_save:
        os.makedirs("pre_processing/mesh", exist_ok=True)
        file_path = f"pre_processing/mesh/rectangular_pml_domain_{ElementType}{ElementOrder}.msh"
        print(f"Malha salva em {file_path}")
        gmsh.write(file_path)
        read_mesh.basic_info()

    # Create mesh Structure Data from gmsh
    mesh_data['cell'] = read_mesh.get_cell_data(MATERIAL)
    mesh_data['nodes'] = read_mesh.get_nodes_data(BOUNDARY)
    mesh_data['edges'] = read_mesh.get_edge_data()

    # Apply physics to the problem
    mesh_data = apply_physics(FINITE_ELEMENT, mesh_data)
    
    gmsh.finalize()

    return mesh_data

# `create_p2_domain()`

In [1]:
def create_p2_domain(FINITE_ELEMENT, BOUNDARY, MATERIAL, DOMAIN_KEY, auto_save=True, view_mesh=False):
    # Parâmetros do elemento finito
    ElementType, ElementOrder = FINITE_ELEMENT
    mesh_data = {}

    # Inicializar o Gmsh
    gmsh.initialize()
    gmsh.model.add("rectangular_pml")
    factory = gmsh.model.occ

    # Dimensões do domínio 
    h = DOMAIN_KEY['h']     # Tamanho do elemento finito
    L = DOMAIN_KEY['L']     # Lado do retângulo externo
    ra = DOMAIN_KEY['ra']   # Raio do furo circular
    x0 = DOMAIN_KEY['x0']   # Lado do retângulo interno
    xa = x0
    xb = x0 + L

    # Criar regiões absorvedoras, omega_PML
    region_i = factory.addRectangle(-xa, -xb, 0, 2*x0, L)
    region_a = factory.addRectangle(-xb, -xb, 0, L, L)
    region_b = factory.addRectangle(xa, -xb, 0, L, L)
    region_ii = factory.addRectangle(xa, -xa, 0, L, 2*x0)
    region_c = factory.addRectangle(xa, xa, 0, L, L)
    region_iii = factory.addRectangle(-xa, xa, 0, 2*x0, L)
    region_d = factory.addRectangle(-xb, xa, 0, L, L)
    region_iv = factory.addRectangle(-xb, -xa, 0, L, 2*x0)

    # Criar região do espaço livre, omega_fs
    region_fs = factory.addRectangle(-xa, -xa, 0, 2*x0, 2*x0)

    # Fragmentar todas as regiões para garantir interfaces conformais
    objectDimTags = [
        (2, region_fs),
        (2, region_i), (2, region_ii), (2, region_iii), (2, region_iv), 
        (2, region_a), (2, region_b), (2, region_c), (2, region_d)
    ]
    
    factory.fragment(objectDimTags, objectDimTags)
    
    # Criar região do espalhador, omega_s
    disk = factory.addDisk(0, 0, 0, ra, ra)

    # Subtrair omega_s de omega_fs
    outDimTags_omega_s, _ = factory.cut([(2, region_fs)], [(2, disk)], removeTool=True)
        
    # Sincronizar após o corte do retângulo interno
    factory.synchronize()

    # Obter o contorno (curva, dim=1) de gamma_s
    DimTags_free_space = gmsh.model.getBoundary(outDimTags_omega_s, oriented=True, recursive=False)

    # TAGs de gamma_s
    TagList_scatterer = [-tag[1] for tag in DimTags_free_space if tag[1] < 0]

    # TAGs de gamma_fs
    tagList_fs = [tag[1] for tag in DimTags_free_space if tag[1] > 0]

    # Adicionar grupos físicos para curvas (Dim=1)
    for i, CurveTagList in enumerate([TagList_scatterer]):
        gmsh.model.addPhysicalGroup(1, CurveTagList, tag=BOUNDARY[i]['tag'], name=BOUNDARY[i]['name'])

    # Adicionar grupos físicos para superfícies (Dim=2)	    
    TagList_surfaces = [region_fs, region_a, region_b, region_c, region_d, region_i, region_ii, region_iii, region_iv]

    for i, SurfaceList in enumerate(TagList_surfaces):
        gmsh.model.addPhysicalGroup(2, [SurfaceList], tag=MATERIAL[i]['tag'], name=MATERIAL[i]['name'])

    # Definir ordem dos elementos
    gmsh.option.setNumber("Mesh.MeshSizeMax", h)
    gmsh.option.setNumber("Mesh.MeshSizeMin", h)
    gmsh.model.mesh.generate(2)
    gmsh.model.mesh.setOrder(ElementOrder)

    # Visualizar a malha no ambiente Gmsh (opcional)
    if view_mesh:
        gmsh.fltk.run()
    
    if auto_save:
        os.makedirs("pre_processing/mesh", exist_ok=True)
        file_path = f"pre_processing/mesh/rectangular_pml_domain_{ElementType}{ElementOrder}.msh"
        print(f"Malha salva em {file_path}")
        gmsh.write(file_path)
        read_mesh.basic_info()

    # Create mesh Structure Data from gmsh
    mesh_data['cell'] = read_mesh.get_cell_data(MATERIAL)
    mesh_data['nodes'] = read_mesh.get_nodes_data(BOUNDARY)
    mesh_data['edges'] = read_mesh.get_edge_data()

    # Apply physics to the problem
    mesh_data = apply_physics(FINITE_ELEMENT, mesh_data)    
    gmsh.finalize()
    return mesh_data

# `create_p3_domain()`

In [ ]:
def create_p3_domain(FINITE_ELEMENT, BOUNDARY, MATERIAL, DOMAIN_KEY, auto_save=True, view_mesh=False):
    # Parâmetros do elemento finito
    ElementType, ElementOrder = FINITE_ELEMENT
    mesh_data = {}

    # Inicializar o Gmsh
    gmsh.initialize()
    gmsh.model.add("rectangular_pml")
    factory = gmsh.model.occ

    # Dimensões do domínio 
    # Dimensões do domínio 
    h = DOMAIN_KEY['h']     # Tamanho do elemento finito
    L = DOMAIN_KEY['L']     # Lado do retângulo externo
    ra = DOMAIN_KEY['ra']   # Raio do furo circular
    x0 = DOMAIN_KEY['x0']   # Lado do retângulo interno
    xa = x0
    xb = x0 + L

    # Criar regiões absorvedoras retangulares, omega_PML
    region_a = factory.addRectangle(-xb, -xb, 0, L, L)
    region_e = factory.addRectangle(-xa, -xb, 0, 2*x0, L)
    region_b = factory.addRectangle(xa, -xb, 0, L, L)
    region_f = factory.addRectangle(xa, -xa, 0, L, 2*x0)
    region_c = factory.addRectangle(xa, xa, 0, L, L)
    region_g = factory.addRectangle(-xa, xa, 0, 2*x0, L)
    region_d = factory.addRectangle(-xb, xa, 0, L, L)
    region_h = factory.addRectangle(-xb, -xa, 0, L, 2*x0)

    # Criar região do espaço livre, omega_fs
    region_fs = factory.addRectangle(-xa, -xa, 0, 2*x0, 2*x0)

    # Criar uma linha de integração
    CutLineTags = factory.addLine(
        factory.addPoint(ra, 0, 0, h),
        factory.addPoint(xb, 0, 0, h))

    # Criar região do espalhador, omega_s
    disk = factory.addDisk(0, 0, 0, ra, ra)
    outDimTags_omega_s, _ = factory.cut([(2, region_fs)], [(2, disk)], removeTool=True)

    # Obter as TAGs das superfícies fragmentadas
    factory.synchronize()
    FragmentedSurfaceTags = [DimTag[1] for DimTag in gmsh.model.getEntities(dim=2)]
    print("TAGs das Superfícies Fragmentadas:", sorted(FragmentedSurfaceTags))

    # Fragmentar todas as regiões para garantir interfaces conformais
    fragmentedObject = [
        (2, region_fs), (2, region_a), (2, region_e), (2, region_b),
        (2, region_f), (2, region_c), (2, region_g), (2, region_d),
        (2, region_h), (1, CutLineTags)
    ]
    print("Pares (Dim, Tag) dos objetos para conformação:\n", fragmentedObject)

    outDimTags, _ = factory.fragment(fragmentedObject, fragmentedObject)
    print("Pares (Dim, Tag) dos objetos conformados:\n", outDimTags)

    # Sincronizar após o corte do retângulo interno
    factory.synchronize()

    # Obter os contorno de fronteira
    LineTags = [DimTag[1] for DimTag in gmsh.model.getEntities(dim=1)]
    print("TAGs dos Contornos:", sorted(LineTags))

    # Obter as TAGs das superfícies
    SurfaceTags = [DimTag[1] for DimTag in gmsh.model.getEntities(dim=2)]
    print("TAGs das Superfícies:", sorted(SurfaceTags))

    # Obter o contorno (curva, dim=1) de gamma_s
    boundary_free_space = gmsh.model.getBoundary(outDimTags_omega_s, oriented=True)

    # Obter o contorno do espalhador e do espaço livre
    ScattererTags = [-DimTag[1] for DimTag in boundary_free_space if DimTag[1] < 0]
    print("Contorno do espalhador:", ScattererTags)

    FreeSpaceTags = [DimTag[1] for DimTag in boundary_free_space if DimTag[1] > 0]
    print("Contorno do espaço livre:", FreeSpaceTags)

    def get_pml_boundary(tag_region, name):
        tags = sorted([DimTag[1] for DimTag in gmsh.model.getBoundary([(2, tag_region)], oriented=False)])
        print(f"Contorno de {name}:", tags)
        return tags

    # Obter o contorno (curva, dim=1) de PML
    pml_a = get_pml_boundary(1, 'PML_a')
    pml_b = get_pml_boundary(3, 'PML_b')
    pml_c = get_pml_boundary(5, 'PML_c')
    pml_d = get_pml_boundary(7, 'PML_d')
    pml_i = get_pml_boundary(2, 'PML_i')
    pml_ii = get_pml_boundary(10, 'PML_ii')
    pml_iii = get_pml_boundary(11, 'PML_iii')
    pml_iv = get_pml_boundary(6, 'PML_iv')
    pml_v = get_pml_boundary(8, 'PML_v')

    # Obter o conjunto de contornos pml
    pml_xy = pml_a + pml_b + pml_c + pml_d
    pml_x = pml_ii + pml_iii + pml_v
    pml_y = pml_i + pml_iv
    pml_list = pml_xy + pml_x + pml_y
    print("Contorno da PML:", pml_list)

    # Obter os contornos internos
    cut_line_pml = list(set(pml_ii) & set(pml_iii))
    cut_line_fs = list(set(LineTags) - set(pml_list) - set(ScattererTags))
    CutLineTags = cut_line_pml + cut_line_fs
    print("Contornos internos:", CutLineTags)

    # Obter o contorno externo da PML
    from collections import Counter
    PmlOuterTags = sorted([key for key, value in Counter(pml_list+FreeSpaceTags).items() if value == 1])
    print("Contorno externo da PML:", PmlOuterTags)

    # Adicionar grupos físicos para curvas (Dim=1)
    print("Grupos físicos de Dim=1")
    PhysicalGroupsDim1 = [ScattererTags, FreeSpaceTags, PmlOuterTags, CutLineTags]
    for i, curves in enumerate(PhysicalGroupsDim1):
        print(BOUNDARY[i]['tag'], curves)
        gmsh.model.addPhysicalGroup(1, curves, tag=BOUNDARY[i]['tag'], name=BOUNDARY[i]['name'])

    # Adicionar grupos físicos para superfícies (Dim=2)    
    print("Grupos físicos de Dim=2")
    PhysicalGroupsDim2 = [[9], [1, 3, 5, 7], [8, 10, 11], [2, 6]]
    for i, surfaces in enumerate(PhysicalGroupsDim2):
        print(MATERIAL[i]['tag'], surfaces)
        gmsh.model.addPhysicalGroup(2, surfaces, tag=MATERIAL[i]['tag'], name=MATERIAL[i]['name'])

    # Definir ordem dos elementos
    gmsh.option.setNumber("Mesh.MeshSizeMax", h)
    gmsh.option.setNumber("Mesh.MeshSizeMin", h)
    gmsh.model.mesh.generate(2)
    gmsh.model.mesh.setOrder(ElementOrder)

    # Visualizar a malha no ambiente Gmsh (opcional)
    if view_mesh:
        gmsh.fltk.run()
    
    if auto_save:
        os.makedirs("pre_processing/mesh", exist_ok=True)
        file_path = f"pre_processing/mesh/rectangular_partial_pml_domain_{ElementType}{ElementOrder}.msh"
        print(f"Malha salva em {file_path}")
        gmsh.write(file_path)
        read_mesh.basic_info()

    # Create mesh Structure Data from gmsh
    mesh_data['cell'] = read_mesh.get_cell_data(MATERIAL)
    mesh_data['nodes'] = read_mesh.get_nodes_data(BOUNDARY)
    mesh_data['edges'] = read_mesh.get_edge_data()

    # Apply physics to the problem
    mesh_data = apply_physics(FINITE_ELEMENT, mesh_data)    
    gmsh.finalize()
    return mesh_data

# `create_p4_domain()`

In [ ]:
def create_p4_domain(FINITE_ELEMENT, BOUNDARY, MATERIAL, GEOMETRY, auto_save=True, view_mesh=False):
    # Parâmetros do elemento finito
    ElementType, ElementOrder = FINITE_ELEMENT
    mesh_data = {}

    # Inicializar o Gmsh
    gmsh.initialize()
    gmsh.model.add("rectangular_pml")
    factory = gmsh.model.occ

    # Dimensões do domínio 
    h = GEOMETRY['h']     # Tamanho do elemento finito
    L = GEOMETRY['L']     # Espessura da PML
    ra = GEOMETRY['ra']   # Raio do espalhador
    x0 = GEOMETRY['x0']   # Fronteira interna da PML
    xa = x0
    xb = x0 + L

    # Criar regiões absorvedoras, omega_PML
    region_i = factory.addRectangle(-xa, -xb, 0, 2*x0, L)
    region_a = factory.addRectangle(-xb, -xb, 0, L, L)
    region_b = factory.addRectangle(xa, -xb, 0, L, L)
    region_ii = factory.addRectangle(xa, -xa, 0, L, 2*x0)
    region_c = factory.addRectangle(xa, xa, 0, L, L)
    region_iii = factory.addRectangle(-xa, xa, 0, 2*x0, L)
    region_d = factory.addRectangle(-xb, xa, 0, L, L)
    region_iv = factory.addRectangle(-xb, -xa, 0, L, 2*x0)

    # Criar região do espaço livre, omega_fs
    region_fs = factory.addRectangle(-xa, -xa, 0, 2*x0, 2*x0)

    # Fragmentar todas as regiões para garantir interfaces conformais
    objectDimTags = [
        (2, region_fs),
        (2, region_i), (2, region_ii), (2, region_iii), (2, region_iv), 
        (2, region_a), (2, region_b), (2, region_c), (2, region_d)
    ]
    
    factory.fragment(objectDimTags, objectDimTags)
    
    # Criar região do espalhador, omega_s
    disk = factory.addDisk(0, 0, 0, ra, ra)

    # Subtrair omega_s de omega_fs
    outDimTags_omega_s, _ = factory.cut([(2, region_fs)], [(2, disk)], removeTool=True)
        
    # Sincronizar após o corte do retângulo interno
    factory.synchronize()

    # Obter o contorno (curva, dim=1) de gamma_s
    DimTags_free_space = gmsh.model.getBoundary(outDimTags_omega_s, oriented=True, recursive=False)

    # TAGs de gamma_s
    TagList_scatterer = [-tag[1] for tag in DimTags_free_space if tag[1] < 0]

    # TAGs de gamma_fs
    tagList_fs = [tag[1] for tag in DimTags_free_space if tag[1] > 0]

    # Adicionar grupos físicos para curvas (Dim=1)
    for i, CurveTagList in enumerate([TagList_scatterer]):
        gmsh.model.addPhysicalGroup(1, CurveTagList, tag=BOUNDARY[i]['tag'], name=BOUNDARY[i]['name'])

    # Adicionar grupos físicos para superfícies (Dim=2)	    
    TagList_surfaces = [region_fs, region_a, region_b, region_c, region_d, region_i, region_ii, region_iii, region_iv]

    for i, SurfaceList in enumerate(TagList_surfaces):
        gmsh.model.addPhysicalGroup(2, [SurfaceList], tag=MATERIAL[i]['tag'], name=MATERIAL[i]['name'])

    # Definir ordem dos elementos
    gmsh.option.setNumber("Mesh.MeshSizeMax", h)
    gmsh.option.setNumber("Mesh.MeshSizeMin", h)
    gmsh.model.mesh.generate(2)
    gmsh.model.mesh.setOrder(ElementOrder)

    # Visualizar a malha no ambiente Gmsh (opcional)
    if view_mesh:
        gmsh.fltk.run()
    
    if auto_save:
        os.makedirs("pre_processing/mesh", exist_ok=True)
        file_path = f"pre_processing/mesh/rectangular_pml_domain_{ElementType}{ElementOrder}.msh"
        print(f"Malha salva em {file_path}")
        gmsh.write(file_path)
        read_mesh.basic_info()

    # Create mesh Structure Data from gmsh
    mesh_data['cell'] = read_mesh.get_cell_data(MATERIAL)
    mesh_data['nodes'] = read_mesh.get_nodes_data(BOUNDARY)
    mesh_data['edges'] = read_mesh.get_edge_data()
    # mesh_data = apply_physics(FINITE_ELEMENT, mesh_data)    
    
    gmsh.finalize()
    return mesh_data

# Computation Domain, $\Omega_c$

## `ez_at_node()`

In [9]:
def ez_at_node(node, ra, series_terms=40):
    # Converter coordenadas cartesianas para coordenadas polares
    ez = 0    
    x, y = node['xg']
    rho, phi = np.sqrt(x**2 + y**2), np.arctan2(y, x)
    for n in range(0, series_terms):
        e_n = 2 if n != 0 else 1
        Jn_ka = jv(n, K0 * ra)
        Hn2_ka = hankel2(n, K0 * ra)
        Hn2_kp = hankel2(n, K0 * rho)
        ez += (-1j) ** n * e_n * (Jn_ka / Hn2_ka) * Hn2_kp * np.cos(n * phi)
    return ez

## `ezh_at_contour()`

In [11]:
def ez_at_contour(mesh_data, contour='BGT'):
    ez = {}
    for key, node in mesh_data['nodes'].items():
        if node['bc']['type'] == contour:
            # Coordenadas cartesianas do nó
            x, y = node['xg']
            
            # Converter coordenadas cartesianas para coordenadas polares
            rho, phi = np.sqrt(x**2 + y**2), np.arctan2(y, x)

            # Converter phi para o intervalo [0, 2*pi]
            if phi < 0:
                phi = 2 * np.pi + phi

            # Analytical solution
            ez[key] = {
                'phi': phi,
                'ez': ez_at_node(node)
            }
    return ez

## `calculate_error()`

In [12]:
def calculate_error(u_exact, u_approx):
    """
    Calcula o erro entre a solução exata e a solução aproximada.

    Parâmetros:
    - u_exact: Array com os valores da solução exata u_ex.
    - u_approx: Array com os valores da solução aproximada u_h.

    Retorna:
    - erro: Valor do erro calculado.
    """
    # Verificar se os arrays têm o mesmo tamanho
    if len(u_exact) != len(u_approx):
        raise ValueError("Os arrays u_exact e u_approx devem ter o mesmo tamanho.")

    # Calcular a soma dos quadrados das diferenças
    N = len(u_exact)
    error = np.sqrt(np.sum((u_exact - u_approx)**2) / N)
    return np.abs(error)

## `plot_coordinates()`

In [13]:
def plot_coordinates(mesh_data):
    pec_coords = []         # Coordenadas de nós do espalhador
    inn_pml_coords = []     # Coordenadas de nós da fronteira interna da PML

    for node in mesh_data['nodes'].values():
        x, y = node['xg']
        if node['bc']['tag'] == 101:
            pec_coords.append((x, y))
        elif node['bc']['tag'] == 102:
            inn_pml_coords.append((x, y))        

    # Conversão para listas separadas de x e y
    pec_x, pec_y = zip(*pec_coords) if pec_coords else ([], [])
    inn_pml_x, inn_pml_y = zip(*inn_pml_coords) if inn_pml_coords else ([], [])

    # Visualização
    plt.figure(figsize=(8, 8))
    plt.scatter(pec_x, pec_y, color='blue', label='Scatterer', marker='o', s=10)
    plt.scatter(inn_pml_x, inn_pml_y, color='red', label='Inner PML', marker='x', s=10)
    plt.axhline(0, color='gray', linewidth=0.5, linestyle='--')
    plt.axvline(0, color='gray', linewidth=0.5, linestyle='--')
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.gca().set_aspect('equal', adjustable='box')
    plt.legend()
    plt.title("Visualização de Nós por Tipo de Condição de Contorno")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.show()

# Physical Domain, $\Omega_{fs}$

## `get_physical_mesh_data()`

In [ ]:
def get_physical_mesh_data(mesh_data):
    mesh_data_copy = copy.deepcopy(mesh_data)

    # Estruturas de dados para a malha física
    physical_mesh_data = {'cell': {}, 'nodes': {}}

    # Filtrar células do domínio físico
    for key, cell in mesh_data_copy['cell'].items():
        if cell['material']['tag'] == 201:
            physical_mesh_data['cell'][key] = cell
            for node in cell['conn']:
                physical_mesh_data['nodes'][node] = mesh_data_copy['nodes'][node]

    # Ordenar as chaves do dicionário 'nodes'
    physical_mesh_data['nodes'] = dict(sorted(physical_mesh_data['nodes'].items()))

    # Recriar conectividade com índices ajustados
    node_index_map = {node: idx+1
                    for idx, node in enumerate(physical_mesh_data['nodes'].keys())}

    # Atualizar a conectividade das células
    for cell in physical_mesh_data['cell'].values():
        cell['conn_physical'] = [node_index_map[node] for node in cell['conn']]

    return physical_mesh_data

## `physical_structured_data()`

In [ ]:
def physical_structured_data(physical_mesh_data):
    # Extrair coordenadas dos nós físicos
    xg_phy = [node['xg'][0] for node in physical_mesh_data['nodes'].values()]
    yg_phy = [node['xg'][1] for node in physical_mesh_data['nodes'].values()]

    # Recriar conectividade com índices ajustados
    conn_py_phy = [[node - 1 for node in cell['conn_physical'][:3]]
                    for cell in physical_mesh_data['cell'].values()]
    
    return xg_phy, yg_phy, conn_py_phy

## `plot_physical_mesh()`

In [ ]:
def plot_physical_mesh(FINITE_ELEMENT, mesh_data, physical_mesh_data, domain='Entire', numbering=False):

    # Extrair estruturas de dados
    ElementType, ElementOrder = FINITE_ELEMENT
    xg_phy, yg_phy, conn_py_phy = physical_structured_data(physical_mesh_data)

    plt.figure(figsize=(8, 6))
    if ElementType == "Triangle":
        # Plotando a malha de elementos finitos
        plt.triplot(xg_phy, yg_phy, conn_py_phy, color='gray')  
 
        # Adicionando os números dos nós e elementos
        if numbering:            
            # Adicionando os números dos nós originais
            for key, node in physical_mesh_data['nodes'].items():
                x, y = node['xg'][0], node['xg'][1]
                plt.scatter(x, y, color='white', edgecolor='black', s=180)
                plt.text(x, y, str(key), color='red', fontsize=8, ha='center', va='center')

            # Adicionando os números dos elementos no centro
            for key, cell in physical_mesh_data['cell'].items():
                xc = np.mean([mesh_data['nodes'][node]['xg'][0] for node in cell['conn']])
                yc = np.mean([mesh_data['nodes'][node]['xg'][1] for node in cell['conn']])
                plt.text(xc, yc, str(key), fontweight='bold', color='black', fontsize=9, ha='center', va='center')

    if not numbering:
        plt.scatter(xg_phy, yg_phy, color='black', s=1, zorder=3)   
        
    # Ajustando rótulos e layout
    plt.xlabel(r'$x$')
    plt.ylabel(r'$y$')
    plt.axis('equal')
    plt.tight_layout()
    
    # Salvando o arquivo no formato SVG
    filepath = graph_results.get_dir(f"pre_processing/pictures/meshed_physical_domain_{ElementType}{ElementOrder}_{domain}Domain.svg")
    plt.savefig(filepath, format="svg")
    plt.close()
    print(f"Arquivo salvo em: {filepath}")

## `solution_at_physical_domain()`

In [ ]:
def solution_at_physical_domain(FINITE_ELEMENT, DOMAIN_KEY, physical_mesh_data, domain='Entire', type='abs'):
    # Extrair estruturas de dados
    ElementType, ElementOrder = FINITE_ELEMENT
    ra = DOMAIN_KEY['ra']
    xg_phy, yg_phy, conn_py_phy = physical_structured_data(physical_mesh_data)

    # Cálculo da solução analítica
    ez = {key: ez_at_node(node, ra)
           for key, node in physical_mesh_data['nodes'].items()}

    # Conversão do dicionário de soluções para lista
    if type in OPERATIONS:
        ez_list = OPERATIONS[type](list(ez.values()))
    else:
        raise ValueError("Tipo de solução inválido. Use 'real', 'imag' ou 'abs'.")

    # Verificação do tipo e ordem do elemento
    if ElementType == "Triangle":
        triangulation = Triangulation(xg_phy, yg_phy, conn_py_phy)
    else:
        raise ValueError("Apenas elementos triangulares ('Triangle') são suportados.")

    # Plot da solução
    plt.figure(figsize=(8, 6))
    plt.tricontourf(triangulation, ez_list, cmap='viridis')
    plt.colorbar(label=r'$|E_z^s(x, y)|$')
    plt.triplot(triangulation, color='gray', alpha=0.5)
    plt.xlabel(r'$x$')
    plt.ylabel(r'$y$')
    plt.tight_layout()
    plt.axis('equal')

    # Salvando o gráfico
    filepath = graph_results.get_dir(f"pos_processing/pictures/analytical_solution_at_physical_domain_{ElementType}{ElementOrder}_{domain}Domain.svg")
    plt.savefig(filepath, format="svg")
    plt.close()
    print(f"Arquivo salvo em: {filepath}")

    return ez

## `fem_solution_at_physical_domain()`

In [ ]:
def fem_solution_at_physical_domain(FINITE_ELEMENT, mesh_data, physical_mesh_data, ezh, domain='Entire', type='abs'):
    # Extrair estruturas de dados
    ElementType, ElementOrder = FINITE_ELEMENT
    xg_phy, yg_phy, conn_py_phy = physical_structured_data(physical_mesh_data)
    
    # Filtrar os nós do domínio físico
    ezh_phy = {key: ezh[key] 
               for key in mesh_data['nodes'] if key in physical_mesh_data['nodes']}

    # Conversão do dicionário de soluções para lista
    if type in OPERATIONS:
        ezh_list = OPERATIONS[type](list(ezh_phy.values()))
    else:
        raise ValueError("Tipo de solução inválido. Use 'real', 'imag' ou 'abs'.")
    
    # Verificação do tipo e ordem do elemento
    if ElementType == "Triangle":
        triangulation = Triangulation(xg_phy, yg_phy, conn_py_phy)
    else:
        raise ValueError("Apenas elementos triangulares ('Triangle') são suportados.")

    # Plot da solução
    plt.figure(figsize=(8, 6))
    plt.tricontourf(triangulation, ezh_list, cmap='viridis')
    plt.colorbar(label=r'$|{Ez}^s_h(x, y)|$')
    plt.triplot(triangulation, color='gray', alpha=0.5)
    plt.xlabel(r'$x$')
    plt.ylabel(r'$y$')
    plt.tight_layout()
    plt.axis('equal')

    # Salvando o gráfico
    filepath = graph_results.get_dir(f"pos_processing/pictures/fem_solution_at_physical_domain_{ElementType}{ElementOrder}_{domain}Domain.svg")
    plt.savefig(filepath, format="svg")
    plt.close()
    print(f"Arquivo salvo em: {filepath}")

    return ezh_phy

# Appendix

## `resume_info()`

In [17]:
def resume_info(ez, ez_h):
    data = []
    for node, ez_value in ez.items():
        ezh_value = ez_h[node]
        abs_ez = abs(ez_value)
        abs_ezh = abs(ezh_value)

        data.append({
            "Node": node,
            "ez": ez_value,
            "ez_h": ezh_value,
            "|ez|": abs_ez,
            "|ez_h|": abs_ezh,
            "Error (%)": (abs_ez - abs_ezh) / abs_ez * 100
        })
    return pd.DataFrame(data)

## `resume_physical_info()`

In [ ]:
def resume_physical_info(mesh_data, ez, ez_h):
    # Identificar nós pertencentes apenas ao domínio físico
    physical_nodes = {
        node for cell in mesh_data['cell'].values()
        if cell['material']['name'] != 'PML'
        for node in cell['conn']
    }

    # Coletar informações apenas para nós físicos
    data = []
    for node in physical_nodes:
        ez_value = ez.get(node, None)
        ezh_value = ez_h.get(node, None)

        # Ignorar nós que não têm valores em ez ou ez_h
        if ez_value is None or ezh_value is None:
            continue

        abs_ez = abs(ez_value)
        abs_ezh = abs(ezh_value)

        # Evitar divisão por zero em erros relativos
        error_percent = (
            (abs_ez - abs_ezh) / abs_ez * 100 if abs_ez != 0 else None
        )

        data.append({
            "Node": node,
            "ez": ez_value,
            "ez_h": ezh_value,
            "|ez|": abs_ez,
            "|ez_h|": abs_ezh,
            "Error (%)": error_percent
        })

    return pd.DataFrame(data)

## `mesh_data_to_table()`

In [18]:
def cell_data_to_table(mesh_data):
    data = []
    for key, cell in mesh_data['cell'].items():
        row = {
            "Cell": key,
            # "Connections": cell['conn'],
            # "Material Tag": cell['material']['tag'],
            # "Material Name": cell['material']['name'],
            # "Rel. Magnetic Permeability": cell['material']['relative_magnetic_permeability'],
            # "Rel. Electric Permittivity": cell['material']['relative_electric_permittivity'],
            # "Source Type": cell['source']['type'],
            # "Source Value": cell['source']['value'],
            "Stiffness Matrix": cell['stiffness_a_value'],
            "Mass Value": cell['mass_a_value'],
            # "ABC Type": cell['abc']['type'],
            # "ABC Connections": cell['abc']['conn_idx']
        }
        data.append(row)

    return pd.DataFrame(data)

Conversão do arquivo Jupyter Notebook para um script Python: ``python -m nbconvert --to script name.ipynb``

Belo Horizonte, Brazil. 2024.  
Adilton Junio Ladeira Pereira - adt@ufmg.br  
&copy; All rights reserved.